[Classification problem](https://www.jianshu.com/p/169dc01f0589)

[code reference](https://blog.csdn.net/qq_29462849/article/details/80640285)

[sampling reference](http://www.dataivy.cn/blog/3-4-解决样本类别分布不均衡的问题/)

The whole idea:
1. consturct three different datasets: original, over-sampling and under-sampling.
2. Try a series of different models: KNN/ SVC/ DecisionTree/ RandomForest/ Adaboost/ GradientBoosting/ GaussianNB/ LinearDiscrimination/ QuadraticDiscriminant
3. Find the best model and best dataset.
4. Improve the model by tuunning and c-v and others.

In [35]:
import pandas as pd

print("Loading the training data...")
x_v0_train = pd.read_csv("X_train.csv", header =0,index_col=0)
x_test = pd.read_csv("X_test.csv",header = 0 ,index_col=0)
y_train = pd.read_csv("y_train.csv",header = 0,index_col=0,names=['id','label'])

Loading the training data...


In [36]:
# have a quick view of the data
x_v0_train.describe()
y_train.groupby('label').size()

label
0.0     600
1.0    3600
2.0     600
dtype: int64

In [33]:
#importing necessary packages
!pip install metrics
!pip install imbalanced-learn

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py bdist_wheel for metrics: started
  Running setup.py bdist_wheel for metrics: finished with status 'done'
  Stored in directory: C:\Users\fgr07\AppData\Local\pip\Cache\wheels\38\e8\55\c4c648451a33a60920318eba4c5735088dbf78dc20d4277db0
  Running setup.py bdist_wheel for pathspec: started
  Running setup.py bdist_wheel for pathspec: finished with status 'done'
  Stored in directory: C:\Users\fgr07\AppData\Local\pip\Cache\wheels\d1\2e\6b\c67842adf3b13ea33ca3fe6a2a963622821ad647db724688e2
Successfully built metrics pathspec


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [37]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE # 过抽样处理库SMOTE
from imblearn.under_sampling import RandomUnderSampler # 欠抽样处理库RandomUnderSampler
from sklearn.svm import SVC #SVM中的分类算法SVC
from imblearn.ensemble import EasyEnsemble # 简单集成方法EasyEnsemble

model_smote = SMOTE() # 建立SMOTE模型对象
x_over_train, y_over_train = model_smote.fit_sample(x_v0_train,y_train) # 输入数据并作过抽样处理

model_RandomUnderSampler = RandomUnderSampler() # 建立RandomUnderSampler模型对象
x_under_train, y_under_train =model_RandomUnderSampler.fit_sample(x_v0_train,y_train) # 输入数据并作欠抽样处理

# 使用SVM的权重调节处理不均衡样本
model_svm = SVC(class_weight='balanced') # 创建SVC模型对象并指定类别权重
model_svm.fit(x_v0_train, y_train) # 输入x和y并训练模型

C:\Users\fgr07\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\fgr07\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
# have a view of the size of resampled data
y_over_train =pd.DataFrame(y_over_train,columns=['label'])
y_over_size=y_over_train.groupby('label').size()

y_under_train =pd.DataFrame(y_under_train,columns=['label'])
y_under_size=y_under_train.groupby('label').size()

print("The over sampling size is ",y_over_size,"\n The under sampling size is ",y_under_size)

The over sampling size is  label
0.0    3600
1.0    3600
2.0    3600
dtype: int64 
 The under sampling size is  label
0.0    600
1.0    600
2.0    600
dtype: int64


In [46]:
from sklearn.model_selection import train_test_split

print("Spleating the dataset...")
X_v0_train, X_v0_val, Y_v0_train, Y_v0_val = train_test_split(x_v0_train,y_train,test_size=0.2)
X_over_train, X_over_val, Y_over_train, Y_over_val = train_test_split(x_over_train,y_over_train,test_size=0.2)
X_under_train, X_under_val, Y_under_train, Y_under_val = train_test_split(x_under_train,y_under_train,test_size=0.2)

Spleating the dataset...


In [ ]:
from sklearn.metrics import accuracy_score, log_loss 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
#from sklearn.metrics import balanced_accuracy_score

classifiers = [ KNeighborsClassifier(3), SVC(kernel="rbf", C=0.025, probability=True), 
                DecisionTreeClassifier(), RandomForestClassifier(),
               AdaBoostClassifier(), GradientBoostingClassifier(), GaussianNB(), 
               LinearDiscriminantAnalysis(), QuadraticDiscriminantAnalysis()]
def ModelsOnDatasets(clfs, datasets, x_train,x_test,y_train,y_test):
    print("The Datasets are: ",datasets)
    # Logging for Visual Comparison 
    #log_cols=["Classifier", "Accuracy", "Log Loss","BMAC"] 
    log_cols=["Classifier", "Accuracy", "Log Loss"]
    log = pd.DataFrame(columns=log_cols) 
    for clf in clfs: 
        clf.fit(x_train, y_train) 
        name = clf.__class__.__name__ 
        print("="*30,'\n', name)
    
        train_predictions = clf.predict(x_test) 
        acc = accuracy_score(y_test, train_predictions) 
        
        #BMAC = balanced_accuracy_score(y_test, train_predictions)
    
        train_predictions = clf.predict_proba(x_test) 
        ll = log_loss(y_test, train_predictions) 
        
        # write the item in the log
        #log_entry = pd.DataFrame([[name, acc*100, ll, BMAC]], columns=log_cols) 
        log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols) 
        log = log.append(log_entry) 
        
    
        print('****Results****') 
        print("Accuracy: {:.4%}".format(acc)) 
        print("Log Loss: {}".format(ll)) 
        print("="*30)
    return log
    

log_under=ModelsOnDatasets(classifiers,"Under Sampling Datasets",X_under_train, X_under_val, Y_under_train, Y_under_val)
log_v0=ModelsOnDatasets(classifiers,"Original Datasets",X_v0_train, X_v0_val, Y_v0_train, Y_v0_val)
log_over=ModelsOnDatasets(classifiers,"Over Sampling Datasets",X_over_train, X_over_val, Y_over_train, Y_over_val)

The Datasets are:  Under Sampling Datasets


C:\Users\fgr07\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


 KNeighborsClassifier
****Results****
Accuracy: 61.1111%
Log Loss: 6.964452958236254


C:\Users\fgr07\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 SVC
****Results****
Accuracy: 53.6111%
Log Loss: 0.8528225091641564
 DecisionTreeClassifier
****Results****
Accuracy: 50.5556%
Log Loss: 17.077506106372503


...

 GradientBoostingClassifier
****Results****
Accuracy: 65.5556%
Log Loss: 0.7473577923553658
 GaussianNB
****Results****
Accuracy: 58.6111%
Log Loss: 12.959001805839732
 LinearDiscriminantAnalysis
****Results****
Accuracy: 51.9444%
Log Loss: 4.814779155606913


C:\Users\fgr07\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


 QuadraticDiscriminantAnalysis
****Results****
Accuracy: 33.3333%
Log Loss: 23.025850929940457
The Datasets are:  Original Datasets


C:\Users\fgr07\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


 KNeighborsClassifier
****Results****
Accuracy: 79.3750%
Log Loss: 3.6072449036338647


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plotLog(log):
    sns.set_color_codes("muted") 
    sns.barplot(x='Accuracy', y='Classifier', data=log, color="b") 

    plt.xlabel('Accuracy %') 
    plt.title('Classifier Accuracy') 
    plt.show() 

    sns.set_color_codes("muted") 
    sns.barplot(x='Log Loss', y='Classifier', data=log, color="g") 

    plt.xlabel('Log Loss') 
    plt.title('Classifier Log Loss') 
    plt.show()

plotLog(log_v0)
plotLog(log_over)
plotLog(log_under)